# Examen teórico-práctico

#### Mauricio Tumalan Castillo
#### A01369288

## Introducción

En este examen teórico-práctico, desarrollaremos un clasificador de texto utilizando redes neuronales. El objetivo principal es identificar y clasificar textos en una de las 20 categorías disponibles en el conjunto de datos "The 20 Newsgroups". Este conjunto de datos es ampliamente utilizado en tareas de clasificación de texto y procesamiento de lenguaje natural (NLP).

Para lograr este objetivo, seguiremos los siguientes pasos:

1. **Carga y exploración de datos**: Utilizaremos la biblioteca `sklearn` para cargar el conjunto de datos y exploraremos su estructura y contenido.
2. **Preprocesamiento de texto**: Aplicaremos técnicas de preprocesamiento de texto, como la eliminación de stopwords y la vectorización de texto utilizando TF-IDF.
3. **División de datos**: Dividiremos el conjunto de datos en subconjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo.
4. **Construcción del modelo**: Crearemos un modelo de red neuronal utilizando `tensorflow` y `keras`.
5. **Entrenamiento y evaluación**: Entrenaremos el modelo con los datos de entrenamiento y evaluaremos su rendimiento utilizando métricas adecuadas.
6. **Visualización de resultados**: Utilizaremos bibliotecas de visualización como `plotly` para visualizar los resultados y las métricas de rendimiento.

A lo largo del notebook, se proporcionará una explicación detallada de cada paso y se documentará el uso de las librerías necesarias para completar el proyecto.

# Documentación de Uso de Librerías

En este notebook, utilizaremos varias librerías de Python para realizar diversas tareas de procesamiento y análisis de datos. A continuación, se describen las librerías que se importarán y su propósito:

1. **numpy**:
   - `np`: Biblioteca fundamental para la computación científica en Python. Proporciona soporte para matrices y operaciones matemáticas de alto nivel.

2. **pandas**:
   - `pd`: Biblioteca para la manipulación y análisis de datos. Proporciona estructuras de datos y herramientas para trabajar con datos estructurados (tablas).

3. **sklearn.datasets**:
   - `fetch_20newsgroups`: Función para cargar el conjunto de datos de 20 Newsgroups, utilizado para tareas de clasificación de texto.

4. **sklearn.feature_extraction.text**:
   - `TfidfVectorizer`: Clase para convertir una colección de documentos de texto en una matriz de características TF-IDF.

5. **sklearn.model_selection**:
   - `train_test_split`: Función para dividir un conjunto de datos en subconjuntos de entrenamiento y prueba.
   - `KFold`: Clase para la validación cruzada K-Fold.
   - `cross_val_score`: Función para evaluar un modelo mediante validación cruzada.

6. **sklearn.preprocessing**:
   - `LabelEncoder`: Clase para convertir etiquetas categóricas en valores numéricos.
   - `label_binarize`: Función para binarizar etiquetas.

7. **tensorflow.keras.utils**:
   - `to_categorical`: Función para convertir vectores de etiquetas en matrices de etiquetas categóricas (one-hot encoding).

8. **nltk**:
   - `stopwords`: Módulo que proporciona una lista de palabras comunes (stopwords) que se pueden excluir del análisis de texto.

9. **tensorflow.keras.models**:
   - `Sequential`: Clase para crear un modelo secuencial en Keras.

10. **tensorflow.keras.layers**:
    - `Dense`: Capa densa (totalmente conectada) en una red neuronal.

11. **scikeras.wrappers**:
    - `KerasClassifier`: Wrapper para usar modelos de Keras con scikit-learn.

12. **plotly.express**:
    - `px`: Biblioteca para la creación de gráficos interactivos.

13. **plotly.graph_objects**:
    - `go`: Biblioteca para la creación de gráficos más complejos y personalizados.

14. **sklearn.metrics**:
    - `confusion_matrix`: Función para calcular la matriz de confusión.
    - `roc_curve`: Función para calcular la curva ROC.
    - `auc`: Función para calcular el área bajo la curva ROC.

Asegúrate de tener instaladas todas las librerías necesarias antes de ejecutar las celdas de código. Puedes instalar las librerías utilizando `pip` si aún no las tienes instaladas.

In [36]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier

import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

from sklearn.metrics import confusion_matrix, roc_curve, auc

## Cargar el dataset

En esta sección, cargamos el conjunto de datos "The 20 Newsgroups" utilizando la función `fetch_20newsgroups` de la biblioteca `sklearn.datasets`. Este conjunto de datos contiene aproximadamente 20,000 documentos de noticias distribuidos en 20 categorías diferentes.

Se le pasa como contexto `None` de las categorías para elegir todas las existentes en el dataset.

In [37]:
# Cargar el dataset

categories = None # Usar todas las categorías
newsgroups = fetch_20newsgroups(subset='all', categories=categories)

## Preprocesamiento de Texto

En esta sección, realizamos el preprocesamiento del texto y la preparación de los datos para el modelo de clasificación. Los pasos incluyen la eliminación de stopwords, donde se utiliza el conjunto de palabras vacías de la librería nltk para eliminar palabras comunes en inglés que no aportan significado relevante, ayudando así al análisis de texto centrándose en las palabras más importantes para la tarea de clasificación.

La vectorización de texto se realizó utilizando TF-IDF, basandose en los beneficios que esta técnica ofrece para convertir los datos textuales en una forma numérica que pueda ser procesada por los algoritmos de aprendizaje automático. El parámetro `max_features=1000` limita el número de características a las 1000 palabras más importantes para reducir dimensionalidad y el coste computacional, manteniendo de igual forma las características más relevantes.

La codificación de etiquetas fue realizada con `LabelEncoder`, convirtiendo las etiquetas de texto (categorías) en valores numéricos. Una vez es realizado esto, se utiliza `to_categorical` para convertir estas etiquetas numéricas en una codificación one-hot, ayudando a los modelos a calcular las probabilidades de pertenencia a cada clase.

El dataset es dividido en un conjunto de entrenamiento y uno de prueba, utilizando el 80% de los datos para entrenamiento y el 20% restante para prueba. El parámetro de `random_state=42` es dado para asegurar la semilla y que la división sea reproducible, facilitando la comparación de resultados en futuras ejecuciones.

In [38]:
# Preprocesamiento de texto
stopwords = set(stopwords.words('english'))

# Vectorizar con TF-IDF
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=1000)
X = vectorizer.fit_transform(newsgroups.data)

# Codificar las etiquetas
encoder = LabelEncoder()
y = encoder.fit_transform(newsgroups.target)
y = to_categorical(y)

# Dividir el dataset en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Visualización de la Distribución de Categorías y Longitud de Textos

En esta sección, visualizamos la distribución de las categorías en el conjunto de datos y la longitud de los textos utilizando gráficos interactivos. Esto nos permite entender mejor la estructura y características del conjunto de datos.

Se utiliza `np.bincount` para contar la frecuencia de cada categoría en el conjunto de datos. Esta función crea un arreglo donde cada posición corresponde a una categoría, siendo su valor el que represente el número de documentos asignados a dicha categoría. Se almacenan los nombres de las categorías con `newsgrous.target_names` para su visualización posterior.

Se genera un histograma utilizando `plotly.express` para visualizar la distribución de las categorías en el dataset. Cada barra del histograma representando una categoría, con su altura representando el número de documentos que pertenecen a la misma. Con esta visualización se puede entender si un conjunto de datos está equilibrado o si algunas categorías tienen más ejemplos que otras, lo cual podría agectar el rendimiento del modelo.

Se calcula la longitud de cada texto del conjunto de datos mediante la función `len()` utilizando un diagrama de caja para visualizar la distribución de longitudes de los textos, pudiendo analizar si la longitud del texto es un factor relevante en la clasificación.

In [39]:
# Distribución de las categorías en el dataset
category_counts = np.bincount(newsgroups.target)
categories = newsgroups.target_names

# Histograma de categorías
fig = px.bar(x=categories, y=category_counts, labels={'x':'Category', 'y':'Count'}, title="Distribution of Categories")
fig.show()

# Diagrama de caja para la longitud de los textos
text_lengths = [len(text) for text in newsgroups.data]
fig_box = px.box(text_lengths, labels={'value':'Text Length'}, title="Distribution of Text Lengths")
fig_box.show()


Se puede determinar que la mayoría de las categorías tienen un número similar de documentos, teniendo así un conjunto de datos equilibrado en cuanto a representación de diferentes clases.

En el diagrama de caja se pueden observar algunos valores atípicos, con documentos significativamente más largos que la mayoría (como el que tiene más de 150,000 caracteres). Aún así se puede observar que la matoría de los textos tiene longitudes consistentes.

## Definir Función para Crear el Modelo

El modelo se define utilizando una arquitectura de red neuronal con una capa de entrada, una oculta y una de salida, todas densamente conectadas, seleccionando las funciones de activación ReLU y Softmax en el siguiente orden:

- **Capa de Entrada**: 100 neuronas con activación ReLU.
- **Capa Oculta**: 50 neuronas con activación ReLU, que permite capturar patrones complejos en los datos.
- **Capa de Salida**: 20 neuronas con activación softmax, adecuada para problemas de clasificación multiclase.

ReLU es elegida sobre una función sigmoide o tahn debido a que tiene la capacidad de solucionar el problema de desvanecimiento del gradiente, el cual es común en las redes neuronales profundas que utilizan las dos funciones mencionadas anteriormente. ReLU es eficiente ya que su derivada es 1 para entradas positivas, evitando así el problema de activaciones constantes en 0, favoreciendo la representación de caracterpisticas no lineales.

En la segunda capa se reduce el número de neuronas en respuesta a una estrategia de disminución gradual de la dimensionalidad, permitiendo que el modelo se enfoque en las características más relevantes a medida que avanza hacia las capas finales.

La tercer capa utiliza la función de activación Softmax debido a que es ideal para las tareas de clasificación multiclase, ya que convierte las salidas de la red en probabilidades (donde la suma de todas es igual a 1). De esta manera, asigna una probabilidad a cada una de las 20 clases posibles, y la clase con mayor probabilidad será predicha por el modelo.


In [40]:
# Definir función para crear el modelo
def create_model(lossSelection):
    model = Sequential()
    model.add(Dense(100, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(20, activation='softmax'))
    model.compile(optimizer='adam', loss=lossSelection, metrics=['accuracy'])
    return model

En el siguiente bloque se llama a la función del bloque anterior, pasando la función de pérdida `categorical_crossentropy`, que es la más adecuada para tareas de clasificación multiclase. Esta función mide la diferencia entre las probabilidades predichas por la red neuronal (generadas con Softmax) y la verdadera distribución de clases (codificada como one-hot). Esta función de pérdida es común en problemas donde la salida tiene múltiples clases, como es el caso del conjunto de datos con 20 categorías.

Se llama a `model_summary()`para generar un resumen detallado de la arquitectura del modelo, el cual nos proporcionará información clave.

In [41]:
# Llamar al modelo
model = create_model("categorical_crossentropy")
# Resumen del modelo
model.summary()

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_39 (Dense)                │ (None, 100)            │       100,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 20)             │         1,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 106,170 (414.73 KB)

 Trainable params: 106,170 (414.73 KB)

 Non-trainable params: 0 (0.00 B)

A partir de este resumen del modelo, se puede concluir que existen tres capas densas como fueron mencionadas anteriormente, teniendo 106,170 parametros entrenables, con 100,100 en la primer capa, 5,050 en la segunda capa y 1,020 en la tercera. No hay parámetros no entrenables, indicando que todos los parámetros son ajustables durante el entrenamiento.

## Entrenamiento del Modelo

El siguiente bloque de código se encarga de entrenar el modelo utilizando el conjunto de datos de entrenamiento y evaluarlo con un conjunto de datos de validación:

1. Se pasan los datos de `X_train` y de `y_train` para que el modelo aprenda a asociar las características del conjunto de entrenamiento con las categorias de entrenamiento.

2. Se ha elegido entrenar el modelo durante 10 épocas, las cuales son definidas en `epochs=10`, cada época pasará todo el conjunto de datos de entrenamiento a través del modelo una vez. Se busca que el modelo tenga suficiente tiempo para aprender patrones en los datos sin llegar al sobreajuste. Las épocas se pueden ajustar en futuros entrenamientos dependiendo de los resultados.

3. El tamaño de lote define cuántos ejemplos se pasan al modelo antes de que se realice una actualización de los pesos, el cual está siendo indicado en `batch_size=64`. Se define como 64 para ofrecer un buen equilibrio entre eficiencia computacional y estabilidad en la actualización de los pesos. Un tamaño más pequeño podría alentizar el entrenamiento, así como uno más grande podría perder capacidad de generalización.

4. Se monitorea el rendimiento fuera del conjunto de entrenamiento al pasar los datos de validación en `validation_data=(X_test, y_test)` en cada época. Esto permite identificar si el modelo comienza a sobreajustarse.

Se almacena el entrenamiento en history, incluyendo métricas de pérdida y precisión en el conjunto de entrenamiento y validación en cada época.

In [42]:
# Entrenamiento del modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3086 - loss: 2.6386 - val_accuracy: 0.6724 - val_loss: 1.2098
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7348 - loss: 0.9999 - val_accuracy: 0.7247 - val_loss: 0.9221
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7936 - loss: 0.7266 - val_accuracy: 0.7430 - val_loss: 0.8413
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8204 - loss: 0.6124 - val_accuracy: 0.7440 - val_loss: 0.8165
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8519 - loss: 0.5196 - val_accuracy: 0.7448 - val_loss: 0.8137
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8634 - loss: 0.4683 - val_accuracy: 0.7493 - val_loss: 0.8190
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8825 - loss: 0.4133 - val_accuracy: 0.7472 - val_loss: 0.8284
Epoch 8/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9016 - loss: 0.3589 - val_accuracy: 0.

Se puede observar como el modelo comienza con una precisión del 30.86% en la primera época, con el progreso terminando en 92.64% para la última, mostrando que el modelo aprende y ajusta sus pesos de forma efectiva en cada época. Igual se puede observar como la perdida va disminuyendo de un 2.63 hasta un 0.28.

La precisión en el conjunto de validación comienza en 67.24% y termina oscilando alrededor del 74% para las últimas, aunque signifique una mejora se puede observar que no continúa aumentando después de la 3er iteración. Se muestra un comportamiento similar en la pérdida en el conjunto de validación, comentanzo en 1.2 y obteniendo su valor más bajo en la 5ta iteracion, con 0.8190.

## Visualización del Proceso de Entrenamiento

En el siguiente bloque se prepararán y visualizarán los datos del resultado del entrenamiento. Se crea un DataFrame de pandas `history_df` para organizar los datos del historial de entrenamiento, facilitando la manipulación y visualización de las métricas a lo largo de las épocas. Las columnas son dadas por los valores de precisión y perdida de `history`.

Se generan gráficas utilizando `plotly.express` para visualizar la evolución de la precisión tanto en el conjunto de entrenamiento como en el de validación, el eje X muestra las épocas mientras el eje Y muestra la precisión alcanzada por época, tanto en entrenamiento como en validación.

Se genera de igual forma una gráfica que muestra cómo va evolucionando la pérdida del modelo en cada época tanto para el conjunto de entrenamiento como para el de validación. mostrando en el eje X la época como en el eje Y la pérdida registrada en cada época.

In [43]:
# Preparar los datos para la visualización
history_df = pd.DataFrame({
    'Epoch': list(range(1, len(history.history['accuracy']) + 1)),
    'Training Accuracy': history.history['accuracy'],
    'Validation Accuracy': history.history['val_accuracy'],
    'Training Loss': history.history['loss'],
    'Validation Loss': history.history['val_loss']
})

# Graficar precisión
fig_acc = px.line(history_df, x='Epoch', y=['Training Accuracy', 'Validation Accuracy'], 
                  title="Training and Validation Accuracy")
fig_acc.show()

# Graficar pérdida
fig_loss = px.line(history_df, x='Epoch', y=['Training Loss', 'Validation Loss'], 
                   title="Training and Validation Loss")
fig_loss.show()

En las gráficas se muestra de manera visual los datos discutidos anteriormente, pudiendo observar como los modelos despues de cierta época se estabilizan en relación a la precisión y error obtenido.

## Evaluación del Modelo y Visualización de la Matriz de Confusión

Después de entrenar el modelo, se evalúan sus predicciones en el conjunto de prueba `X_test`, teniendo así el llamado a `model.predict(X_ṭest)` para generar las probabilidades predichas para cada categoría con la capa Softmax en la salida. Se utiliza `np.argmax()` para convertir estas probabilidades en las etiquetas de clase finales, seleccionando la categoría con la probabilidad más alta. `y_true`se refiere a las etiquetas reales del conjunto de prueba `y_test` también convertidas con `np.argmax()` para que coincidan con el formato de las predicciones.

Se crea la matriz de confusión para evaluar el modelo de clasificación, permitiendo ver con claridad en qué categorias el modelo se desempeña bien y en cuales tiende a confundirse, obteniendo una visión más detallada que solo observar la precisión general.

In [44]:
# Evaluar el conjunto de prueba para obtener predicciones
y_pred = np.argmax(model.predict(X_test), axis=1)  # Predicciones del modelo
y_true = np.argmax(y_test, axis=1)  # Etiquetas verdaderas

# Obtener la matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Crear el heatmap con Plotly
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=categories,  # Nombres de las categorías en los ejes
    y=categories,
    colorscale='Blues',
    hoverongaps=False))

# Añadir detalles a la gráfica
fig_cm.update_layout(
    title='Confusion Matrix',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    xaxis_nticks=36,  # Controla el número de divisiones en el eje
    yaxis_nticks=36,
)

# Mostrar el gráfico
fig_cm.show()

118/118 ━━━━━━━━━━━━━━━━━━━━ 0s 931us/step


Se puede observar en la gráfica que la mayoría de los valores se encuentran en la diagonal principal, indicando que el modelo está realizando predicciones correctas para la mayoría de las clases. Se puede concluír que el modelo tiene buen rendimiento global, teniendo algunas categorías que muestran cierta confusión. Esto se podría mejorar mediante la recolección de más datos, ajuste de hiperparámetros, o técnicas de regularización para mejorar la capacidad del modelo para diferenciar mejor las categorías similares.

# Creación de módelo con arquitectura alternativa

Este modelo alternativo utiliza una arquitectura de red neuronal más profunda en comparación a la arquitectura anterior, con más capas y de tamaños más grandes. A continuación se detallan las capas y sus neuronas:

- **Capa de Entrada**: 256 neuronas con activación ReLU.
- **Capa Oculta 1**: 128 neuronas con activación ReLU.
- **Capa Oculta 2**: 64 neuronas con activación ReLU.
- **Capa de Salida**: 20 neuronas con activación Softmax, adecuada para problemas de clasificación multiclase.

El aumento de neuronas en la primera capa permite que el modelo capture más características complejas desde el inicio. Este tipo de arquitectura sería más útil cuando se tiene un conjunto de datos con muchas características o patrones compejos que requieren mayor poder de representación.

Se agrega una tercera capa para refinir aún más las características aprendidas. Al reducir gradualmente el número de neuronas, el modelo se va enfocando en las características más importantes para la clasificación.

In [48]:
# Crear el modelo alternativo
model_alt = Sequential()
model_alt.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model_alt.add(Dense(128, activation='relu'))
model_alt.add(Dense(64, activation='relu'))
model_alt.add(Dense(20, activation='softmax'))

# Compilar el modelo
model_alt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_alt = model_alt.fit(X_train, y_train)# Crear el modelo alternativo
model_alt = Sequential()
model_alt.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model_alt.add(Dense(128, activation='relu'))
model_alt.add(Dense(64, activation='relu'))
model_alt.add(Dense(20, activation='softmax'))

# Compilar el modelo
model_alt.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_alt = model_alt.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Convertir el historial de entrenamiento en un DataFrame
history_df = pd.DataFrame(history_alt.history)
history_df['epoch'] = history_df.index + 1  # Añadir la columna de épocas

# Visualización de la precisión con Plotly
fig_acc = go.Figure()

# Añadir la precisión del entrenamiento
fig_acc.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['accuracy'],
    mode='lines',
    name='Training Accuracy'
))

# Añadir la precisión de la validación
fig_acc.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['val_accuracy'],
    mode='lines',
    name='Validation Accuracy'
))

# Configuración del gráfico
fig_acc.update_layout(
    title='Training and Validation Accuracy',
    xaxis_title='Epoch',
    yaxis_title='Accuracy',
)

# Mostrar la gráfica de precisión
fig_acc.show()

# Visualización de la pérdida con Plotly
fig_loss = go.Figure()

# Añadir la pérdida del entrenamiento
fig_loss.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['loss'],
    mode='lines',
    name='Training Loss'
))

# Añadir la pérdida de la validación
fig_loss.add_trace(go.Scatter(
    x=history_df['epoch'],
    y=history_df['val_loss'],
    mode='lines',
    name='Validation Loss'
))

# Configuración del gráfico
fig_loss.update_layout(
    title='Training and Validation Loss',
    xaxis_title='Epoch',
    yaxis_title='Loss',
)

# Mostrar la gráfica de pérdida
fig_loss.show()

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



472/472 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.3886 - loss: 2.1043
Epoch 1/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.3417 - loss: 2.3023 - val_accuracy: 0.6859 - val_loss: 0.9921
Epoch 2/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7552 - loss: 0.7908 - val_accuracy: 0.7260 - val_loss: 0.8746
Epoch 3/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8225 - loss: 0.5876 - val_accuracy: 0.7353 - val_loss: 0.8634
Epoch 4/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8614 - loss: 0.4676 - val_accuracy: 0.7385 - val_loss: 0.8665
Epoch 5/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8871 - loss: 0.3808 - val_accuracy: 0.7459 - val_loss: 0.8923
Epoch 6/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9119 - loss: 0.2958 - val_accuracy: 0.7469 - val_loss: 0.9478
Epoch 7/10
236/236 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9412 - loss: 0.2245 - val_accuracy: 0.7393 - val_loss: 0.9991
Epoch 8/10
236/236 ━━━━━━━━━━

La precisión en el conjunto de entrenamiento comenzó en un 34.17% en la primera época y alcanzó el 98.09% en la décima. Esto indica que el modelo está aprendiendo de manera efectiva y ajustando sus pesos a medida que avanza el entrenamiento. Teniendo igual la precisión en el conjunto de validación comenzando en un 68% y alcanzando un máximo de 74.59% en la quinta época, fluctuando a 74% en las últimás epocas.

El modelo alternativo muestra una mejora significativa en la precisión del entrenamiento, de igual forma estancandose como el primer modelo. De igual forma se podrían mejorar los datos con técnicas de regularización como en el otro modelo.

# KerasClassifier y K-Fold Cross-Validation

La implementación de K-Fold Cross-Validation con KerasClassifier ayuda a validar el rendimiento del modelo de manera más rigurosa y proporciona una evaluación más confiable de su capacidad de generalización. Se usa `KerasClassifier`, lo que permite envolver un modelo de Keras dentro de un clasificador de `scikit-learn` para hacer uso de sus funciones como la validación cruzada y el ajuste de hiperparámetros. el parametro `build_fn` recibe la función que construye el modelo, en este caso `create_model()`, que ha sido previamente definida. Para este caso, se utiliza sparse_categorical_crossentropy como función de pérdida, ya que `newsgroups.target` contiene etiquetas en formato entero. El modelo se entrena durante 10 épocas con un tamaño de lote de 64. Se indica verbose=0 para evitar la salida detallada durante el entrenamiento.

`K-Fold` se utiliza para crear una validación cruzada con 5 particiones, dividiendo así el conjunto de datos en 5 grupos, donde en cada iteración uno de estos grupos se utiliza como conjunto de prueba y los otros cuatro como conjunto de entrenamiento. Con shuffle=True se asegura que los datos se barajen antes de dividirse en las diferentes particiones.

`cross_val_score` realiza la validación cruzada utilizando el clasificador de Keras y los datos de entrada X junto con las etiquetas de `newsgroups.target`. Se aplican 5 iteraciones de validación cruzada.

In [46]:
# Usar KerasClassifier y K-Fold cross-validation
estimator = KerasClassifier(build_fn=create_model("sparse_categorical_crossentropy"), epochs=10, batch_size=64, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, newsgroups.target, cv=kfold)

print(f"K-Fold Cross-Validation Results: {results.mean():.2f} ± {results.std():.2f}")

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning:

Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning:

``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.

/home/mtumalan/Desktop/Repos/Clases/ConcentracionIA/venv/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:713: UserWarning:

Skipping variable loading for optimizer 'ada

K-Fold Cross-Validation Results: 0.75 ± 0.00


Se puede observar en el resultado que el rendimiento promedio del modelo a través de 5 particiones es de 75% de precisión con una desviación estandar de 0.00, indicando que el modelo tiene un desempeño consistente, aunque el rendimiento general es moderado.

# ROC-AUC

Las curvas ROC representan la relación entre la tasa de verdaderos positivos y falsos positivos para cada clase, el AUC es una métrica que va del 0 a 1, donde un valor cercano a 1 indica una buena capacidad del modelo para distinguir entre clases.

Se convierten las etiquetas a formato binario para calcular las curvas ROC multiclase. Para cada clase `i` se calculan las tasas de falsos positivos `fpr[i]` y verdaderos positivos `tpr[i]` utilizando la función `roc_curve`. Luego se calcula el área bajo la curva con la función `auc` que mide la capacidad del modelo para separar correctamente las clases.

Se añaden las curvas ROC usando `go.Scatter` en un bucle, generando línea por categoría. El AUC correspondiente se muestra en la leyenda de cada clase.

In [49]:
# Convertir las etiquetas a binario
y_bin = label_binarize(newsgroups.target, classes=range(20))
n_classes = y_bin.shape[1]

# Calcular la curva ROC para cada categoría
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_bin[:, i], model.predict(X)[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Graficar la curva ROC AUC para cada clase
fig_roc = go.Figure()

# Añadir las curvas ROC y el AUC para cada categoría
for i in range(n_classes):
    fig_roc.add_trace(go.Scatter( # Curva ROC para la categoría i
        x=fpr[i], y=tpr[i],
        mode='lines',
        name=f'{categories[i]} (AUC = {roc_auc[i]:.2f})'
    ))

# Configuración del gráfico
fig_roc.update_layout(
    title='ROC Curves for Each Category',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
)

# Mostrar la gráfica
fig_roc.show()

589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 971us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 891us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 886us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 832us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 851us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 773us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 769us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 820us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 802us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step
589/589 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step


De esta gráfica se puede observar que la mayoría de las categorías presentan un AUC cercano o igual a 1.00, lo que indica que el modelo tiene una capacidad discriminatoria muy alta entre clases. Las curvas ROC para todas las clases están bastante cerca de la esquina superior izquierda del gráfico, lo que indica una alta tasa de verdaderos positivos y una baja tasa de falsos positivos en la mayoría de las clases.

# Conclusiones

El modelo de red neuronal ha demostrado un rendimiento bueno en términos de clasificación multiclase, con un AUC muy alto para todas las clases y una precisión que se puede considerar alta en general. El modelo está bien ajustado para este problema, y con algunos ajustes adicionales, podría optimizarse aún más para alcanzar un rendimiento superior en datos no vistos.